# Coding Assignment 4

CS 598 Practical Statistical Learning

2023-11-06

UIUC Fall 2023

**Authors**
* Ryan Fogle
    - rsfogle2@illinois.edu
    - UIN: 652628818
* Sean Enright
    - seanre2@illinois.edu
    - UIN: 661791377

**Contributions**

Part I:
- Ryan contributed to implementing the E-step, Sean contributed to refactoring and completely implementing the EM algorithm.


Part II:
- Sean implemented the Baum-Welch Algorithm, Ryan implemented the Viterbi Algorithm. 

## Part 1: Gaussian Mixtures

In [1]:
import numpy as np
import pandas as pd

### Define Functions

Here we implement the EM algorithm for a $p$-dimensional Gaussian misxture model with $G$ classes.

In the E-step, we calculate the responsibility matrix $r_{nk}$, which represents the probability of a hidden state belonging to a particular class, given the observed data and current Gaussian distribution parameters.

$$
p(z_n = k \mid x_n, \theta^{old}) =
r_{nk} =
\frac{\pi_k \, \mathcal{N}(x_n \mid \mu_k, \Sigma_k)}
     {\Sigma_j \pi_j \, \mathcal{N}(x_n \mid \mu_j, \Sigma_j)}
$$

where $\mathcal{N}$ is defined as
$$
\mathcal{N}(x \mid \mu, \Sigma) =
\frac{1}
     {(2\pi)^{p/2} \, |\Sigma|^{1/2}} \,
     \textrm{exp}[-\frac{1}{2}(x-\mu)^{T} \, |\Sigma|^{-1} (x-\mu)]
$$

and $\pi_k$ is the mixture weights for the $k$ classes, $\mu_k$ is the mean of each class and $\Sigma$ is the shared covariance matrix.

In the M-step, we update $\pi_k$, $\mu_k$ and $\Sigma$.

$$\pi_k = \frac{1}{N}\sum_n r_{nk}$$

$$\mu_k^{new} = \frac{\sum_n r_{nk} \, x_n} {\sum_n r_{nk}}$$

$$
\Sigma = \sum_k \pi_k \frac{\sum_n r_{nk} (x_n - \mu_k^{new}) (x_n - \mu_k^{new})^T}
                           {\sum_n r_{nk}}
$$

In [2]:
def Estep(x: pd.DataFrame, G: int, pi: np.ndarray, mu: pd.DataFrame, sigma: pd.DataFrame):
    """EM algorithm expectation step. Here we estimate the latent variables based on the previous
    estimates of theta to build a responsibility matrix.

    Args:
        x (pd.DataFrame): Data matrix, (n, p)
        G (int): Number of classes
        pi (np.ndarray): Mixing weights, (G,)
        mu (pd.DataFrame): Mean values for each class, (p, G)
        sigma (pd.DataFrame): Shared covariance matrix, (p, p)
    
    Returns:
        np.ndarray: The responsibility matrix of shape (n, G)
    """
    resp = np.zeros((x.shape[0], G))
    for k in range(G):
        resp[:, k] = pi[k] * multivariate_normal_density(x, mu.iloc[:, k], sigma)
    return resp / resp.sum(axis=1).reshape(-1, 1)

def multivariate_normal_density(x: pd.DataFrame, mu_k: pd.DataFrame, sigma: pd.DataFrame):
    """Evaluate multivariate normal probability density.
    This is used in the E-step and in the log-likelihood calculation.

    Args:
        x (pd.DataFrame): data, shape (n, p)
        mu_k (pd.DataFrame): mean for a given class, shape (p,)
        sigma (pd.DataFrame): covariance matrix, shape (p, p)

    Returns:
        np.ndarray: n-dimensional probability densities
    """
    A_mu = (x.T - mu_k.values.reshape(-1, 1)).to_numpy()
    exponent = - 0.5 * np.multiply(A_mu, np.linalg.inv(sigma) @ A_mu).sum(axis=0)
    return 1 / (2 * np.pi * np.sqrt(np.linalg.det(sigma))) * np.exp(exponent)

def Mstep(x: pd.DataFrame, G: int, resp: np.ndarray, sigma: pd.DataFrame):
    """EM algorithm maximization step.

    Args:
        x (np.ndarray): Data matrix, (n, p)
        G (int): Number of classes
        resp (np.ndarray): Responsibility matrix, (n, G)
        sigma (pd.DataFrame): Shared covariance matrix, (p, p)
    
    Returns:
        pi_new (np.ndarray): Updated mixing weights, (G,)
        mu_new (pd.DataFrame): Updated mean values per dimension, (p, G)
        sigma_new (pd.DataFrame): Updated covariance matrix, (p, p)
    """
    n = x.shape[0]
    # Pi
    pi_new = resp.sum(axis=0) / n
    # Mu
    mu_new = (x.T @ resp) / resp.sum(axis=0)
    # Sigma
    sigma_new = sigma.copy()
    sigma_new.values[:, :] = 0
    for k in range(G):
        tmp = x.T - mu_new.values[:, k].reshape(-1, 1)
        tmp = tmp.to_numpy()
        sigma_new += pi_new[k] * tmp @ np.diag(resp[:, k]) @ tmp.T / resp[:, k].sum()
    return pi_new, mu_new, sigma_new

def loglik(x: pd.DataFrame, G: int, pi: np.ndarray, mu: pd.DataFrame, sigma: pd.DataFrame):
    """Calculate log likelihood, given distribution parameters.

    Args:
        x (pd.DataFrame): Input data, shape (n, p)
        G (int): Number of classes
        pi (np.ndarray): Mixing weights, shape (G,)
        mu (pd.DataFrame): Distribution means for each class, shape (p, G)
        sigma (pd.DataFrame): Shared covariance matrix, shape (p, p)

    Returns:
        float: log likelihood
    """
    ll = np.zeros(x.shape[0])
    for k in range(G):
        ll += pi[k] * multivariate_normal_density(x, mu.iloc[:, k], sigma)
    return np.log(ll).sum()

def myEM(data: pd.DataFrame, G: int, prob: np.ndarray,
         mean:pd.DataFrame, Sigma: pd.DataFrame, itmax: int):
    """Main EM algorithm

    Args:
        data (pd.DataFrame: Input data, shape (n, p)
        G (int): Number of classes
        prob (np.ndarray): Mixing weights, shape (G,)
        mean (pd.DataFrame): Distribution means for each class, shape (p, G)
        Sigma (pd.DataFrame): Shared covariance matrix, shape (p, p)
        itmax (int): Number of EM iterations to perform

    Returns:
        (np.ndarray, pd.DataFrame, pd.DataFrame, float): probability vector, means, covariance and
                                                         log-likelihood
    """
    for _ in range(itmax):
        resp = Estep(data, G, prob, mean, Sigma)
        prob, mean, Sigma = Mstep(data, G, resp, Sigma)
        ll = loglik(data, G, prob, mean, Sigma)
    return prob, mean, Sigma, ll   

### Testing EM Algorithm for Gaussian Mixture Model

In [3]:
# Load in data
data = pd.read_csv('faithful.dat', header=0, sep='\s+')
data.head()

,eruptions,waiting
1,3.600,79
2,1.800,54
3,3.333,74
4,2.283,62
5,4.533,85


#### Case 1: G=2

The two-class case.

In [4]:
G = 2
n = data.shape[0]
p1 = 10 / n
p2 = 1 - p1
mu1 = data.iloc[:10, :].mean(axis=0)
mu2 = data.iloc[10:, :].mean(axis=0)

sigma = 1 / n * (
           (data.iloc[:10, :].T - mu1.to_numpy().reshape(-1, 1)) @
           (data[:10].T - mu1.to_numpy().reshape(-1, 1)).T + \
           (data.iloc[10:, :].T - mu2.to_numpy().reshape(-1, 1)) @
           (data[10:].T - mu2.to_numpy().reshape(-1, 1)).T
        )

pi = np.array((p1, p2)) # Shape (G,)
mu = pd.DataFrame({"0": mu1, "1": mu2}) # Shape (p, G)

prob, mean, Sigma, ll = myEM(data, G, pi, mu, sigma, 20)
print("Case G=2")
print(f"prob\n{prob}\n\nmean\n{mean}\n\nSigma\n{Sigma}\n\nloglik\n{ll:.3f}\n")

Case G=2
prob
[0.04297883 0.95702117]

mean
                   0          1
eruptions   3.495642   3.487430
waiting    76.797892  70.632059

Sigma
           eruptions     waiting
eruptions   1.297936   13.924336
waiting    13.924336  182.580092

loglik
-1289.569



#### Case 2: G=3

The three-class case.

In [5]:
G = 3
p1 = 10 / n
p2 = 20 / n
p3 = 1 - p1 - p2
mu1 = data.iloc[:10, :].mean(axis=0)
mu2 = data.iloc[10:30, :].mean(axis=0)
mu3 = data.iloc[30:, :].mean(axis=0)
sigma = 1 / n * (
           (data.iloc[:10].T - mu1.to_numpy().reshape(-1, 1)) @
           (data.iloc[:10].T - mu1.to_numpy().reshape(-1, 1)).T + \
           (data.iloc[10:30].T - mu2.to_numpy().reshape(-1, 1)) @
           (data.iloc[10:30].T - mu2.to_numpy().reshape(-1, 1)).T + \
           (data.iloc[30:].T - mu3.to_numpy().reshape(-1, 1)) @
           (data.iloc[30:].T - mu3.to_numpy().reshape(-1, 1)).T
        )

pi = np.array((p1, p2, p3)) # Shape (G,)
mu = pd.DataFrame({"0": mu1, "1": mu2, "2": mu3}) # Shape (p, G)

prob, mean, Sigma, ll = myEM(data, G, pi, mu, sigma, 20)
print(f"prob\n{prob}\n\nmean\n{mean}\n\nSigma\n{Sigma}\n\nloglik\n{ll:.3f}\n")

prob
[0.04363422 0.07718656 0.87917922]

mean
                   0          1          2
eruptions   3.510069   2.816167   3.545641
waiting    77.105638  63.357526  71.250848

Sigma
           eruptions     waiting
eruptions   1.260158   13.511538
waiting    13.511538  177.964191

loglik
-1289.351



## Part II: HMM

### Baum-Welch Algorithm

**E-step**

Forward probabilities, $\alpha_t(i) = p_{\theta}(x_1, \ldots , x_t, Z_t = i)$, are updated recursively as follows:

$$\alpha_1(i) = w(i) B(i, x_1)$$

$$\alpha_{t+1}(i) = \sum_j \alpha_t(j) A(j,i) B(i, x_{t+1})$$

Backward probabilities, $\beta_t(i) = p_{\theta}(x_{t+1}, \ldots,  x_n \mid Z_t = i)$ are also updated recursively:

$$\beta_n(i) = 1$$
$$\beta{t}(i) = \sum_j A(i, j) B(j, x_{t+1}) \beta_{t+1}(j)$$

Finally, $\alpha$ and $\beta$ are used, along with the transition and emission matrices, to estimate the probability of transitioning between each pair of hidden states, given the observed data, $\gamma_t(i, j) = p_{\theta}(Z_t = i, Z_{t+1} = j \mid x)$. It is calculated as follows.

$$\gamma_t(i,j) = \alpha_t(i) A(i, j) B(j, x_{t + 1}) \beta_{t+1}(j)$$


**M-step**

$w$ is not updated for this assignment.

The updated transition matrix, $A^*$ is calculated by
$$A^*(i, j) = \frac{\sum_{t=1}^{n-1}\gamma_t(i,j)}{\sum_{j'} \sum_{t=1}^{n-1} \gamma_t(i, j)}$$

To update B, we first need to compute the marginal probability for $\gamma$. There are two approaches to marginalizing $\gamma$ over the hidden states, depending on the time value.

For $t < n$, $\gamma_t(i)$ we use the formula below.

$$P(Z_t=i \mid x) = \sum_{j=1}^{m_z} P(Z_t=i, Z_{t+1} = j \mid x) = \sum_{j=1}^{m_z} \gamma_t(i j)$$

For $t = n$, $\gamma_t(i)$ we use:

$$P(Z_t=i \mid x) = \sum_{j=1}^{m_z} P(Z_{t-1}=j, Z_t = i \mid x) = \sum_{j=1}^{m_z} \gamma_{t-1}(j, i)$$

With the marginal probability of $\gamma$, the updated emission matrix, $B^*$ can be calculated:

$$B^*(i, l) = \frac{\sum_{t:x_t = l} \gamma_t(i)} {\sum_t \gamma_t(i)}$$

In [6]:
def BW_onestep(data: np.ndarray, mx: np.ndarray, mz: np.ndarray,
               w: np.ndarray, A: np.ndarray, B: np.ndarray):
    """Perform one iteration of the Baum-Welch algorithm, improving the estimates of
       the transition probability and emission distribution matrices.

    Args:
        data (np.ndarray): Observations
        mx (int): Count of distinct values X can take
        mz (int): Count of distinct values Z can take
        w (np.ndarray): An mz-by-1 probability vector representing the initial distribution for Z1.
        A (np.ndarray): The mz-by-mz transition probability matrix that
                        models the progression from Zt to Zt+1
        B (np.ndarray): The mz-by-mx emission probability matrix,
                        indicating how X is produced from Z

    Returns:
        (np.ndarray, np.ndarray): Updated A and B matrices
    """
    n = data.shape[0]
    
    # E-step
    # ==========================================================

    # Forward algorithm
    # Alpha is an mz-by-T forward probability matrix
    alpha = np.empty((mz, n))
    alpha[:, 0] = np.multiply(w, B[:, data[0]])
    for t in range(n - 1):
        alpha[:, t + 1] = (A.T @ alpha[:, t]) * B[:, data[t + 1]]
    
    # Backward algorithm
    # Beta is an mz-by-T backwards probability matrix
    beta = np.empty((mz, n))
    beta[:, n - 1] = 1
    for t in np.arange(n - 2, -1, step = -1):
        beta[:, t] = A @ (B[:, data[t + 1]] * beta[:, t + 1])

    # Gamma
    gamma = np.empty((mz, mz, n - 1))
    for t in range(n - 1):
        for j in range(mz):
            gamma[:, j, t] = alpha[:, t] * A[:, j] * B[j, data[t + 1]] * beta[j, t + 1]

    # M-step
    # ==========================================================

    # Update A
    A = gamma.sum(axis=2) # Sum over time
    A /= A.sum(axis=1).reshape(-1, 1)

    # Update B
    # Marginalized gamma: mz-by-n
    gamma_marginal = np.empty((mz, n))
    gamma_marginal[:, :n - 1] = gamma.sum(axis=1)
    gamma_marginal[:, n - 1] = gamma[:, :, n - 2].sum(axis=0)
    for l in range(mx):
        B[:, l] = gamma_marginal[:, data == l].sum(axis=1) / gamma_marginal.sum(axis=1)
    return A, B

def myBW(data: np.ndarray, mx: int, mz: int, w: np.ndarray,
         A: np.ndarray, B: np.ndarray, itmax: int):
    """Perform the Baum-Welch algorithm for the Hidden Markov Model to estimate
       the transition probability and emission distribution matrices.

    Args:
        data (np.ndarray): Observations
        mx (int): Count of distinct values X can take
        mz (int): Count of distinct values Z can take
        w (np.ndarray): An mz-by-1 probability vector representing the initial distribution for Z1.
        A (np.ndarray): The mz-by-mz transition probability matrix that
                        models the progression from Zt to Zt+1
        B (np.ndarray): The mz-by-mx emission probability matrix,
                        indicating how X is produced from Z
        itmax (int): Maximum number of EM step iterations to perform
    """
    # Convert range of X values fron [1, 3] to  [0, 2] to facilitate indexing in Python
    data = data - 1
    for _ in range(itmax):
        A, B = BW_onestep(data, mx, mz, w, A, B)
    return A, B

#### Testing Implementation of Baum-Welch algorithm

In [7]:
data = pd.read_csv('coding4_part2_data.txt', header=None).to_numpy().flatten()

# Establish possible observations and number of latent states
mx = np.unique(data).shape[0] # Unique X values
mz = 2 # Given in instructions

# Initialize transition probability and emission distribution matrices
w = np.array((0.5, 0.5))
A = np.full((2, 2), 0.5)
B = np.row_stack([np.array([1, 3, 5]) / 9,
                  np.array([1, 2, 3]) / 6])

# Perform Baum-Welch to find estimates of A and B
A, B = myBW(data, mx, mz, w, A, B, 100)
print(f"A: the {mz}-by-{mz} transition matrix\n\n{A}\n\n"
      f"B: the {mz}-by-{mx} emission matrix\n\n{B}\n")

A: the 2-by-2 transition matrix

[[0.49793938 0.50206062]
 [0.44883431 0.55116569]]

B: the 2-by-3 emission matrix

[[0.22159897 0.20266127 0.57573976]
 [0.34175148 0.17866665 0.47958186]]



### Viterbi Algorithm

In [8]:
def myViterbi(data: np.ndarray, mx: int, mz: int, w: np.ndarray,
         A: np.ndarray, B: np.ndarray, itmax: int):
    """Perform the Viterbi Algorithm to output the most likely latent sequence considering 
        the data and the MLE of the parameters.

    Args:
        data (np.ndarray): Observations
        mx (int): Count of distinct values X can take
        mz (int): Count of distinct values Z can take
        w (np.ndarray): An mz-by-1 probability vector representing the initial distribution for Z1.
        A (np.ndarray): The mz-by-mz transition probability matrix that
                        models the progression from Zt to Zt+1
        B (np.ndarray): The mz-by-mx emission probability matrix,
                        indicating how X is produced from Z
        itmax (int): Maximum number of EM step iterations to perform
    """

    # Perform Baum-Welch to find estimates of A and B
    A, B = myBW(data, mx, mz, w, A, B, itmax)

    # put all valus on log-scale
    w = np.log(w)
    A = np.log(A)
    B = np.log(B)

    # initialize additional parameters
    n = data.shape[0]
    delta = np.zeros((mz, n))
    Z = np.zeros(n, dtype=int)

    # subtract 1 from data, python is indexed by 0 as the start.  
    data = data - 1

    # set initial delta value
    delta[:, 0] = w + B[:, data[0]]

    # update for delta
    for idx in range(n - 1):
        delta[:, idx + 1] = np.max(A + delta[:, idx].reshape(-1,1), axis=0) + B[:, data[idx + 1]]
    # print(delta.T)
    
    # find optimal Z value. 
    Z[n-1] = np.argmax(delta[:, n-1])
    for idx in range(n-1, 0, -1):
        Z[idx - 1] = np.argmax(delta[:, idx-1] + A[:, Z[idx]])

    # add one at the end to match output
    return Z + 1

#### Testing Implementation of the Viterbi Algorithm

In [9]:
data = pd.read_csv('coding4_part2_data.txt', header=None).to_numpy().flatten()

# Establish possible observations and number of latent states
mx = np.unique(data).shape[0] # Unique X values
mz = 2 # Given in instructions

# Initialize transition probability and emission distribution matrices
w = np.array((0.5, 0.5))
A = np.full((2, 2), 0.5)
B = np.row_stack([np.array([1, 3, 5]) / 9,
                  np.array([1, 2, 3]) / 6])

# Load in valid Z values for comparison
Z_valid = []
with open('Coding4_part2_Z.txt', 'r') as f:
    Z_valid = np.array(f.read().strip().split(' ')).astype(int)

# Run Viterbi algorithm
Z = myViterbi(data, mx, mz, w, A, B, 100)

# Output results
print('================== Z Valid ========================\n')
print(Z_valid, '\n')
print('================== Z Calculated ===================\n')
print(Z, '\n')

================== Z Valid ========================

[1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 2 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1
 1 1 1 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 1 1 1 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 1 1 1 1 1
 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1] 

================== Z Calculated ===================

[1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 2 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1
 1 1 1 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 1 1 1 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 1 1 1 1 1
 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1] 



We verify that our output is identical to the expected output.

In [10]:
print('\nZ Valid == Z Calc:', np.array_equal(Z, Z_valid))


Z Valid == Z Calc: True
